# Navajo - Dirty Startup

#### Packages

In [1]:
import json
from math import pi, ceil, sin, floor
from scipy.optimize import root
import numpy as np
import os.path
import sys
sys.path.append("../")
import simulate
from scipy.optimize import curve_fit

from water_properties import density, viscosity, conductivity, heat_capacity
from water_at_saturation_properties import saturation_temperature, vapour_density, vapour_total_compressibility,vaporization_enthalpy

from simplified_simulation import *

## Inputs e parameters

In [2]:
filename = "navajo"
suffix = "ds" #dirty startup

#### Constants

In [3]:
g = 9.80665 # m/s2 - gravitational acceleration
pi = pi # pi

#### Water Inlet

In [4]:
m = 16360 # kg/s cooling Water
Tin = 29 + 273.15 # K
Pin = 200000 # Pa

#### Pipe Information

In [5]:
v_pipe = 5.0 # m/s
L_pipe = 50 # m
water_density=density(Tin, Pin, simplified = True)
water_viscosity = viscosity(Tin, Pin, simplified = True)
D_pipe = 2 * (m / (pi * water_density * v_pipe )) ** 0.5
print("Pipe diameter is {} m ".format(D_pipe))
print("Pipe lenght is {} m ".format(L_pipe))

Pipe diameter is 2.0453534779846856 m 
Pipe lenght is 50 m 


#### Heat Exchanger Information

In [6]:
Do = 0.0222 # m external diameter
Di = Do - 2*0.0007 # m external diameter
n_tubes = 25036
arrange = "triangle"
#arrange = "square"

L = 18.3 # m tube length
W = 1.8 * 2 # m bundle width
ep = 5e-6 # m - roughness
kwall = 11.8 # W / ( m * K )
pitch = 0.0286 # m
Rows = 30
row_pitch = pitch
tubes_per_width = n_tubes/Rows
ResF= 0 # K/W
ResF_to_film= 4.8e-5 # K/W ###############CLEAN

Vext = L * W ** 2 # shell volume in m3

print("Number of tubes per row are ", tubes_per_width)
print("Number of rows is {number:.{digits}f}".format(number=Rows,digits=0))
print("Number of tubes is {number:.{digits}f}".format(number=Rows*tubes_per_width,digits=0))

Number of tubes per row are  834.5333333333333
Number of rows is 30
Number of tubes is 25036


In [7]:
N_wetters = []
row_index = np.linspace(1,Rows,num=Rows)
for i in row_index:
    if arrange == "triangle":
        N_wetters.append(ceil(i/2)-1)
    else:
        N_wetters = row_index-1
N_wetters = np.array(N_wetters)
fNtub = ((N_wetters+1) ** (5/6) - N_wetters ** (5/6))

In [8]:
# Fouling
Alc=300
pH=10.5
Ca=49.42

# Fouling
kf = 2.19 # W / ( m * K ) (Calcium carbonate)

rhof = 2.71e3 # kg/m3
Df = Di * np.exp(- (ResF_to_film* (pi * Di * L)) * 2 * kf / Di)
lf = 1e6*(Di-Df)/2  # um fouling thickness

mf = (Di ** 2 - Df ** 2) * rhof / (4 * Di) + 1e-3
Rf = ResF * L / (pi * Di) 

print("Wall internal diameter is {number:.{digits}f}".format(number=Di,digits=4))
print("Fouling internal diameter is {number:.{digits}f}".format(number=Df,digits=6))
print("Fouling thickness is {number:.{digits}f} um".format(number=1e6*(Di-Df)/2,digits=2))
print("Fouling deposit mass is {number:.{digits}f} kg/m2".format(number=mf,digits=6))

Wall internal diameter is 0.0208
Fouling internal diameter is 0.020550
Fouling thickness is 124.95 um
Fouling deposit mass is 0.337574 kg/m2


#### Operational Conditions

In [9]:
# Expected
Tout_mean_expected = 45 + 273.15
Pvap_expected = 13136# Pa
Tvap_expected = 51.07 + 273.15
Pout_expected = 177000
Pout = 177000 # Pa
Tout = Tout_mean_expected * np.ones(Rows) # K

In [10]:
Tm_expected = 0.5 * (Tin + Tout_mean_expected) # K - mean cooling water temperature
Pm_expected = 0.5 * (Pin + Pout) # Pa - mean cooling water pressure
water_heat_capacity = heat_capacity(Tm_expected, Pm_expected, simplified = True)

exhaust_steam_density = vapour_density(Pvap_expected, simplified = True)
exhaust_vaporization_heat = vaporization_enthalpy(Pvap_expected, simplified = True)

Q_expected = water_heat_capacity * m * (Tout_mean_expected - Tin)
kvap_expected = Q_expected/exhaust_vaporization_heat


print("Expected heat duty to {number:.{digits}f} MW.".format(number=Q_expected*1e-6, digits=2))
print("Expected steam mass flowrate {number:.{digits}f} kg/s.".format(number=kvap_expected, digits=2))
print("The exhaust steam density is equal to {number:.{digits}f} kg/m3."\
      .format(number=exhaust_steam_density, digits=5))
print("The exhaust steam vaporization heat is equal to {number:.{digits}f} J/kg."\
      .format(number=exhaust_vaporization_heat, digits=0))
#vaporization_enthalpy(Pvap, simplified=True)
#print(Pvap, exhaust_vaporization_heat, vaporization_enthalpy(Pvap, simplified=True))

Expected heat duty to 1093.77 MW.
Expected steam mass flowrate 459.69 kg/s.
The exhaust steam density is equal to 0.08763 kg/m3.
The exhaust steam vaporization heat is equal to 2379375 J/kg.


#### Simulation specification

In [11]:
# Set
Pvap = 13030 # Pa
Tvap = saturation_temperature(Pvap, simplified = True)
#print(Pvap, Tvap, saturation_temperature(Pvap, simplified=True))

## Simplified model

In [12]:
Tout = Tout_mean_expected * np.ones(Rows) # K
Tf_in = 1*Tin + 0*Tout + 0*Tvap
Tf_out = 0*Tin + 1*Tout + 0*Tvap
Ti_in = 1*Tin + 0*Tout + 0*Tvap
Ti_out = 0*Tin + 1*Tout + 0*Tvap
To_in = 0.5*Tin + 0*Tout + 0.5*Tvap
To_out = 0*Tin + 0.5*Tout + 0.5*Tvap
m_tube = 0*Tout + m/n_tubes
x0 = [Pout_expected]

for i in range(0, Rows):
     x0 +=[Tout[i], Tf_in[i], Tf_out[i], Ti_in[i], Ti_out[i], To_in[i], To_out[i], m_tube[i]]

In [13]:
args = (m, Tvap, Tin, Pin, Pvap, fNtub, Rows, tubes_per_width, kf, kwall, ep, L, Df, Do, Di, Ca, Alc, pH)
        
x = solve(x0, *args)

results = calculate_results(x,*args)


 -- ROW 0 --
Restotal is 0.000235 W/(K)
The row external convection coefficient is 14801.55 W/(K*m2)
The row internal convection coefficient is 9805.51 W/(K*m2)
The last row calculated internal resistance x length is 0.000086 K/W
The last row calculated external resistance x length is 0.000053 K/W
LMTD is 11.11 K
The heat rate per tube is 47241.81 W
U*A per tube is 4250.53 W/K
Reswall is 0.000048 K/W
Resint is 0.000086 K/W
Resext is 0.000053 K/W
Tf is 315.66 K
Ti is 318.36 K
To is 321.07 K
Tvap is 324.06 K
Cooling water inlet temperature is 302.15 K
Cooling water outlet temperature is 319.43 W/K
Water mean velocity is 1.99 m/s.
Inlet Pressure is 200000.00 Pa.
Outlet Pressure :  163687.87775248912
Tube mass flowrate (kg/s) :  0.6543719835542475
ResFouling (K/W) :  4.800000000000005e-05
dmfdt (kg/m2day) :  3.290472092486912e-08

 -- ROW 29 --
Restotal is 0.000300 W/(K)
The row external convection coefficient is 6723.42 W/(K*m2)
The row internal convection coefficient is 9687.05 W/(K*m2)

In [14]:
# Calculating final results
Q_total = 0
kvap_calc = 0
Tout_mean = 0
Pout_mean = Pout
m_tube_aux = 0
for i in range(0, Rows):
    Q_total += results[i]["Q"]*tubes_per_width
    kvap_calc += results[i]["kvap"]
    Tout_mean += results[i]["Tout"]*results[i]["m_tube"]
    m_tube_aux += results[i]["m_tube"]
Tout_mean = Tout_mean/m_tube_aux
    
print("The total steam flowrate is {number:.{digits}f} kg/s".format(number=kvap_calc, digits=2))
print("The total heat load of condensation is {number:.{digits}f} MW".format(number=1e-6*Q_total, digits=2))
print("The cooling water CALCULATED mean outlet temperature is {number:.{digits}f} K".format(number=Tout_mean,digits=2))
print("The cooling water EXPECTED mean outlet temperature is {number:.{digits}f} K".format(number=Tout_mean_expected,digits=2))
print("Cooling Water outlet temp deviation is {number:.{digits}f} %".format(number=(Tout_mean-Tout_mean_expected)/(Tout_mean_expected-273.15)*100,digits=2))
print("Steam temp deviation is {number:.{digits}f} %".format(number=(Tvap-Tvap_expected)/(Tvap_expected - 273.15)*100,digits=2))

The total steam flowrate is 459.36 kg/s
The total heat load of condensation is 1093.18 MW
The cooling water CALCULATED mean outlet temperature is 318.14 K
The cooling water EXPECTED mean outlet temperature is 318.15 K
Cooling Water outlet temp deviation is -0.02 %
Steam temp deviation is -0.32 %


## Daetools Simulations 

In [15]:
Nelements = 15

In [16]:
def prepare_initial_estimate(results, key1, key2):
    return np.array([list(np.linspace(result[key1], result[key2], Nelements)) for result in results]).T.tolist()

#### Initial estimate for Steady State with fixed To

In [17]:
data = {

    "node_B": {
        "kind": "node",
        "module": "models.source",
        "class": "Source",
        "steady_state": True,
        "specifications": {
            "w": m,
        },
        "parameters": {
            "Text": Tin,
            "Pext": Pin,
            "x": 0.0,
            "y": 0.0,
            "z": 0.0
        },
        "initial_guess": {
            "w": m,
            "P": Pin,
            "T": Tin
        }
    },   
    
        "condenser": {
        "kind": "edge",

        "module": "models.carbfilmed_fixed_external_temperature_tube_arrange",
        "class": "CarbfilmedFixedExternalTemperatureTubeArrange",

        #"module": "models.tube_arrange",
        #"class": "TubeArrange",

        "from": "node_B",
        "to": "node_C",
        "domains": {
            "x": {
                "initial": 0.0,
                "final": 1.0,
                "N": Nelements
            },
            "y": {
                "N": Rows
            },

        },
        "states": {
            'stnFouling': "Static",
        },
        "parameters": {
            "Di": Di,
            "tetha": 0.0,
            "L": L,
            "epw": ep,
            "Klb": 0.0,
            "Kub": 0.0,
            "Npipes": n_tubes/Rows,
            "kwall": kwall,
            "Do": Do,
            "ResF": 0.0,
            "mfi": mf,
            "Alc": Alc,
            "pH": pH,
            "Ca": Ca,



        },
        "steady_state": True,
        "specifications": {
            "To": prepare_initial_estimate(results, "To_in", "To_out"),
        },
        "initial_guess": {
            "D": Df,
            "v": prepare_initial_estimate(results, "v", "v"),
            "k": prepare_initial_estimate(results, "m_tube", "m_tube"),
            "T": prepare_initial_estimate(results, "Tin", "Tout"),
            "P": prepare_initial_estimate(results, "Pin", "Pout"),
            "hint": prepare_initial_estimate(results, "hint", "hint"),
            "Ti": prepare_initial_estimate(results, "Ti_in", "Ti_out"),
            "mf": mf,
            "Rf": Rf,
            "phid": prepare_initial_estimate(results, "phid", "phid"),
            "phir": prepare_initial_estimate(results, "phir", "phir"),
        }
    },
    
        "node_C": {
        "kind": "node",
        "module": "models.sink",
        "class": "Sink",
        "steady_state": True,
        "specifications": {
            "P": Pout,
        },
        "parameters": {
            "Text": Tout_mean,
            "Pext": Pout,
            "x": 0.0,
            "y": 0.0,
            "z": 0.0
        },
        "initial_guess": {
            "w": m,
            "P": Pout,
            "T": Tout_mean

        }
    }, 
    
}

with open("{}_{}_pre.json".format(filename, suffix), 'w') as outfile:
    json.dump(data, outfile, indent=4)
    
simulate.main(
    name=filename,
    input="{}_{}_pre.json".format(filename, suffix),
    output="{}_{}_pre.out.json".format(filename, suffix),
    reporting_interval=1,
    time_horizon=1,
    relative_tolerance=1e-1,
    MaxStep=1,
    MaxNumSteps=1000000,
)

Creating the system...        
The system created successfully.
 0.00% ETA: [00.0s]Starting the initialization of the system... Done.
 0.00% ETA: [00.0s]Saving Initialization Values
Integrating from [0] to [1] ...
 0.00% ETA: [00.0s]                              
 100.00% ETA: [00.0s]The simulation has finished successfully!
 100.00% ETA: [00.0s]                              
 100.00% ETA: [00.0s]--------------------------------------------------------------
 100.00% ETA: [00.0s]                             Time (s)   Rel.time (%)    Count
 100.00% ETA: [00.0s]--------------------------------------------------------------
 100.00% ETA: [00.0s]Simulation:                   
 100.00% ETA: [00.0s]  Initialization                2.316              -        -
 100.00% ETA: [00.0s]  Solve initial                16.863          31.79        -
 100.00% ETA: [00.0s]  Integration                  36.185          68.21        -
 100.00% ETA: [00.0s]  Integ. + solve init.         53.048         10

#### Initial Estimate with complete model

In [18]:
data2 = data.copy()

data2["condenser"] = {
        "kind": "edge",

        "module": "models.carbfilmed_external_film_cond_tube_arrange",
        "class": "CarbfilmedExternalFilmCondensationTubeArrange",

        "from": "node_B",
        "to": "node_C",
        "domains": {
            "x": {
                "initial": 0.0,
                "final": 1.0,
                "N": Nelements
            },
            "y": {
                "N": Rows
            },

        },
        "states": {
            'stnRegulator': 'Closed',
            'stnShellPressure': 'Variable',
            'stnFouling': "Static",
        },

        "parameters": {
            "Di": Di,
            "tetha": 0.0,
            "L": L,
            "epw": ep,
            "Klb": 0.0,
            "Kub": 0.0,
            "Npipes": n_tubes/Rows,
            "kwall": kwall,
            "Do": Do,
            "PextH": 100000,
            "PextSP": -1,
            "Pext0": Pvap,
            "Vext": Vext,
            "kvap": kvap_calc,
            "fNtub": list(fNtub),
            "ResF": 0.0,
            "mfi": mf,
             "Alc": Alc,
            "pH": pH,
            "Ca": Ca,



        },
        "steady_state": True,
        "specifications": {
            #"kcond": 0.0,
            #"Pext": Pvap,
            #"Text": Tvap,
            #"wext": 0.0,
            #"To": prepare_initial_estimate(results, "To_in", "To_out"),
        },
        "initial_guess": {
            "D": Df,
            "v": prepare_initial_estimate(results, "v", "v"),
            "k": prepare_initial_estimate(results, "m_tube", "m_tube"),
            "T": prepare_initial_estimate(results, "Tin", "Tout"),
            "P": prepare_initial_estimate(results, "Pin", "Pout"),
            "hint": prepare_initial_estimate(results, "hint", "hint"),
            "hext": prepare_initial_estimate(results, "hext", "hext"),
            "Pext": Pvap,
            "Text": Tvap,
            "Ti": prepare_initial_estimate(results, "Ti_in", "Ti_out"),
            #"Qout": 0.0,
            "To": prepare_initial_estimate(results, "To_in", "To_out"),
            "kcond": kvap_calc,
            #"wext": 0.0,
            "mf": mf,
            "Rf": Rf,
            "phid": prepare_initial_estimate(results, "phid", "phid"),
            "phir": prepare_initial_estimate(results, "phir", "phir"),
        }
}

with open("{}_{}.json".format(filename, suffix), 'w') as outfile:
    json.dump(data2, outfile, indent=4)
    
simulate.main(
    name=filename,
    input="{}_{}.json".format(filename, suffix),
    output="{}_{}.out.json".format(filename, suffix),
    initial_condition="{}_{}_pre.out.json".format(filename, suffix),
    reporting_interval=1,
    time_horizon=1,
    relative_tolerance=1e-3,
    MaxStep=1,
    MaxNumSteps=1000000,

)

with open("{}_{}.out.json".format(filename, suffix)) as f:
    json_data=f.read()
output = json.loads(json_data)

Tout_sim = output["node_C.T"]["Values"][-1]
Tvap_sim = output["condenser.Text"]["Values"][-1]

print("Simulated Steam Pressure {number:.{digits}f} Pa".format(
    number=output["condenser.Pext"]["Values"][-1],digits=6)
     )
print("Simulated Steam Temperature {number:.{digits}f} oC".format(
    number=Tvap_sim-273.15,digits=2)
     )
print("Simulated Cooling Water Inlet Temperature {number:.{digits}f} oC".format(
    number=output["node_B.T"]["Values"][-1]-273.15,digits=2)
     )
print("Simulated Cooling Water Outlet Temperature {number:.{digits}f} oC".format(
    number=Tout_sim-273.15,digits=2)
     )
print("Simulated Steam flowrate {number:.{digits}f} kg/s".format(
    number=output["condenser.kvap"]["Values"][-1],digits=2)
     )
print("Simulated Steam Pressure {number:.{digits}f} Pa".format(
    number=sum(np.array(output["condenser.Qtotal"]["Values"][-1]))/1e6,digits=2)
     )

print("Cooling Water outlet temp deviation is {number:.{digits}f} %".format(number=(Tout_sim-Tout_mean_expected)/(Tout_mean_expected-273.15)*100,digits=2))
print("Steam temp deviation is {number:.{digits}f} %".format(number=(Tvap_sim-Tvap_expected)/(Tvap_expected - 273.15)*100,digits=2))

Creating the system...        
The system created successfully.
 0.00% ETA: [00.0s]Starting the initialization of the system... Done.
 0.00% ETA: [00.0s]Saving Initialization Values
Integrating from [0] to [1] ...
 0.00% ETA: [00.0s]                              
 100.00% ETA: [00.0s]The simulation has finished successfully!
 100.00% ETA: [00.0s]                              
 100.00% ETA: [00.0s]--------------------------------------------------------------
 100.00% ETA: [00.0s]                             Time (s)   Rel.time (%)    Count
 100.00% ETA: [00.0s]--------------------------------------------------------------
 100.00% ETA: [00.0s]Simulation:                   
 100.00% ETA: [00.0s]  Initialization                3.119              -        -
 100.00% ETA: [00.0s]  Solve initial                35.791          31.13        -
 100.00% ETA: [00.0s]  Integration                  79.194          68.87        -
 100.00% ETA: [00.0s]  Integ. + solve init.        114.985         10